In [1]:
import cv2
import face_recognition
import pickle
import time

In [2]:
image_file = "image/marathon_03.jpg"
encoding_file = "encodings.pickle"
unknown_name = "Unknown"
model_method = "cnn"

In [11]:
def detectAndDisplay(image):
    # 시작시간 저장
    start_time = time.time()
    
    # 읽어온 이미지를 RGB모드로 변환
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    #이미지에 있는 모든 얼굴을 인식하여 boxes에 저장
    boxes = face_recognition.face_locations(rgb, model=model_method)
    # 얼굴인식 형태의 인코딩을 거쳐 encodings 리스트에 저장
    encodings = face_recognition.face_encodings(rgb, boxes)
    
    # 표시할 이름들의 리스트 생성
    # (Son, Tedy, Unknown 셋 중 하나의 값들로 저장될 리스트) 
    names = []
    
    # 읽어온 이미지에서 인식된 얼굴 개수만큼 반복 실행
    for encoding in encodings:
        # 반복실행 중, 현재 얼굴(encoding)과 모듈의 얼굴들(data)을 비교합니다
        matches = face_recognition.compare_faces(data["encodings"], encoding)
        
        # 현재얼굴의 최초 이름은 "Unknown"으로 저장
        name = unknown_name
        
        # 얼굴이 매칭되면, (아래) 
        if True in matches:
            # True 자리의 인덱스값들 저장
            matchedidxs = [i for (i,b) in enumerate(matches) if b]
            counts={}
            # 매칭된 얼굴의 이름을 추출 저장
            for i in matchedidxs:
                # True였던 자리의 인덱스를 이용하여 이름 추출
                name = data["names"][i]
                # 현재이름의 출현회수를 counts라는 dictionary에 각 name을 키 값으로
                # 회수를 카운트합니다
                counts[name] = counts.get(name, 0) + 1
            # 위에서 집계한 True 값의 개수가 가장 많았던 이름을 name에 업데이트
            name = max(counts, key=counts.get)
        # 결정된 "Unknown" or "Son" or "Tedy"를 append
        names.append(name)
    
    # 박스데이터와 해당 이름들을 조합한 데이터 개수만큼 반복실행
    for((top, right, bottom, left), name) in zip(boxes, names):
        # 텍스트가 입력될 좌표 계산
        y = top -15 if top -15 > 15 else top + 15
        # 섹 지정(녹색), 라인 굵기 2
        color = (0,255,0)
        line=2
        # 이름이 Unknown과 같으면, (아래)
        if(name == unknown_name):
            color = (0,255,255) # 빨간색
            line = 1 # 라인굵기 1
            name = '' # 이름 : 빈칸으로
        cv2.rectangle(image, (left, top), (right, bottom), color, line)
        cv2.putText(image, name, (left, y), cv2.FONT_HERSHEY_SIMPLEX, 0.75, \
                                                                    color, line)
    
    # 종료시간 저장
    end_time = time.time()
    # 경과시간 계산 & 출력
    process_time = end_time - start_time
    print("===A frame took {:.3f} seconds".format(process_time))
    
    cv2.imshow("Recognition", image)

In [13]:
data = pickle.loads(open(encoding_file, "rb").read())
image = cv2.imread(image_file)
detectAndDisplay(image)

cv2.waitKey(0)
cv2.destroyAllWindows()

===A frame took 75.005 seconds
